In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gensim
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('/content/drive/My Drive/DS 102 Bootcamp - Finans Proje 2/KodluyoruzFinans/materials/trmodel', binary=True)

In [4]:
word2vec.doesnt_match(["banka","para","hisse","araba"])

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'araba'

In [5]:
# Import everything that will be used
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
import gensim
import sys
import numpy as np
import pandas as pd
import string
import logging
import random
import matplotlib.pyplot as plt

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Train Word2Vec model with gensim on the dataset

df1 = pd.read_csv("/content/drive/My Drive/DS 102 Bootcamp - Finans Proje 2/KodluyoruzFinans/materials/Deneme Sevdası/akbankzemberek.csv")
df2 = pd.read_csv("/content/drive/My Drive/DS 102 Bootcamp - Finans Proje 2/KodluyoruzFinans/materials/Deneme Sevdası/DF.csv")


In [6]:
df1.label.fillna(0.0,inplace=True)

In [7]:
df1.label.dtype

dtype('float64')

In [8]:
sentences = df1.content

In [9]:
num_words = 1650

In [10]:
#kaç kelime var
a = set()
for i in sentences:
  for k in i.split():
    a.add(k)
len(a)

2113

In [11]:
from keras.preprocessing.text import Tokenizer

tokenizer=Tokenizer(num_words=1650)

In [12]:
tokenizer.fit_on_texts(sentences)

In [13]:
token = tokenizer.texts_to_sequences(sentences)

In [14]:
token[0]

[129, 1022, 522, 141, 1412, 729, 523, 56, 524, 213, 848, 39, 56, 141, 574, 363]

In [15]:
num_tokens = [len(tokens) for tokens in token]

# Convert list to numpy array
num_tokens = np.array(num_tokens)
num_tokens.shape

(1860,)

In [16]:
max_tokens = np.mean(num_tokens) + 2*np.std(num_tokens) #returns float
max_tokens = int(max_tokens)
max_tokens

25

In [17]:
# How many tokens are smaller than max_tokens?
np.sum(num_tokens < max_tokens) / len(num_tokens) * 100  # output: 96%. which means we will only lose info in 4%.

94.89247311827957

In [18]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(token, maxlen=max_tokens)

In [19]:
print(x_train_pad.shape)

(1860, 25)


In [20]:
idx = tokenizer.word_index

# in idx, key value pair like 'çok': 1. But we want it reverse.
inverse_map = dict(zip(idx.values(), idx.keys()))
first_five = {k: inverse_map[k] for k in sorted(inverse_map.keys())[:5]}
first_five

{1: 'bank', 2: 've', 3: 'akbank', 4: 'ol', 5: 'için'}

In [21]:
def convert_tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = ' '.join(words)
    return text

In [22]:
from keras.utils import to_categorical
encoded = to_categorical(df1.label,3)

In [23]:
encoded.shape

(1860, 3)

In [24]:
len(encoded)

1860

In [26]:
# w2vModel = word2vec.KeyedVectors.load_word2vec_format('trmodel', binary=True)

AttributeError: ignored

In [31]:
word2vec.syn0.

AttributeError: ignored

In [32]:
embedding_layer = Embedding(input_dim=word2vec.syn0.shape[0], output_dim=word2vec.syn0.shape[1], weights=[word2vec.syn0], 
                            input_length=x_train_pad.shape[1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [33]:
model = Sequential()

model.add(embedding_layer)

# 3-layered LSTM
model.add(LSTM(units=16, return_sequences=True))
# model.add(Dropout(0.2))
model.add(LSTM(units=8, return_sequences=True))
# model.add(Dropout(0.2))
model.add(LSTM(units=4, return_sequences=False))
# model.add(Dropout(0.2))
# Dense layer: aka fully connected layer. Consists of one neuron.
# model.add(Dense(1, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))

In [34]:
from tensorflow.python.keras.optimizers import Adam
# Adam optimizer
optimizer = Adam(lr=1e-3)

In [35]:
# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 400)           164982800 
_________________________________________________________________
lstm (LSTM)                  (None, 25, 16)            26688     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 8)             800       
_________________________________________________________________
lstm_2 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dense (Dense)                (None, 3)                 15        
Total params: 165,010,511
Trainable params: 165,010,511
Non-trainable params: 0
_________________________________________________________________


In [37]:
# epoch -> how many times we are going to train our data.
# batch_size -> feeding size
model.fit(x_train_pad, encoded, epochs=10, batch_size=32)

Epoch 1/10
59/59 [==============================] - 88s 1s/step - loss: 1.0705 - accuracy: 0.4468
Epoch 2/10
59/59 [==============================] - 83s 1s/step - loss: 1.0318 - accuracy: 0.4656
Epoch 3/10
59/59 [==============================] - 82s 1s/step - loss: 0.9733 - accuracy: 0.5290
Epoch 4/10
59/59 [==============================] - 82s 1s/step - loss: 0.8818 - accuracy: 0.6022
Epoch 5/10
59/59 [==============================] - 82s 1s/step - loss: 0.7919 - accuracy: 0.6457
Epoch 6/10
59/59 [==============================] - 83s 1s/step - loss: 0.7023 - accuracy: 0.7145
Epoch 7/10
59/59 [==============================] - 82s 1s/step - loss: 0.6382 - accuracy: 0.7516
Epoch 8/10
59/59 [==============================] - 82s 1s/step - loss: 0.5655 - accuracy: 0.7806
Epoch 9/10
59/59 [==============================] - 84s 1s/step - loss: 0.5053 - accuracy: 0.8081
Epoch 10/10
59/59 [==============================] - 86s 1s/step - loss: 0.4599 - accuracy: 0.8355


In [43]:
df1.content[25]

'akbank ın yıl uluslararası piyasa türki ye ol güve tazele milyon dol sendikasyon kredi global capital tarafından türki de yıl işlem seçil '

In [39]:
x_train_pad_2 = x_train_pad.reshape((-1,1,25))

In [40]:
x_train_pad_2.shape

(1860, 1, 25)

In [ ]:
x_train_pad_3 = x_train_pad.reshape((-1,25,1))

In [42]:
model.predict(x_train_pad_2[25]),encoded[25]

(array([[0.96842885, 0.00581077, 0.02576045]], dtype=float32),
 array([1., 0., 0.], dtype=float32))

In [ ]:
  x_train_pad[1]

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, 
                    weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')